In [1]:
import numpy as np
from gym import spaces
from gym.utils import seeding

import math
import matplotlib
import random
import matplotlib.pyplot as plt

import mayavi.mlab
from mayavi import mlab

float_dtype = np.float32
int_dtype = np.int32

In [2]:
def Bresenham3D(x1, y1, z1, x2, y2, z2):
    ListOfPoints = [[x1, y1, z1]]
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    dz = abs(z2 - z1)
    if x2 > x1:
        xs = 1
    else:
        xs = -1
    if y2 > y1:
        ys = 1
    else:
        ys = -1
    if z2 > z1:
        zs = 1
    else:
        zs = -1

    # Driving axis is X-axis"
    if dx >= dy and dx >= dz:
        p1 = 2 * dy - dx
        p2 = 2 * dz - dx
        while x1 != x2:
            x1 += xs
            if p1 >= 0:
                y1 += ys
                p1 -= 2 * dx
            if p2 >= 0:
                z1 += zs
                p2 -= 2 * dx
            p1 += 2 * dy
            p2 += 2 * dz
            ListOfPoints.append([x1, y1, z1])

    # Driving axis is Y-axis"
    elif dy >= dx and dy >= dz:
        p1 = 2 * dx - dy
        p2 = 2 * dz - dy
        while y1 != y2:
            y1 += ys
            if p1 >= 0:
                x1 += xs
                p1 -= 2 * dy
            if p2 >= 0:
                z1 += zs
                p2 -= 2 * dy
            p1 += 2 * dx
            p2 += 2 * dz
            ListOfPoints.append([x1, y1, z1])

    # Driving axis is Z-axis"
    else:
        p1 = 2 * dy - dz
        p2 = 2 * dx - dz
        while z1 != z2:
            z1 += zs
            if p1 >= 0:
                y1 += ys
                p1 -= 2 * dz
            if p2 >= 0:
                x1 += xs
                p2 -= 2 * dz
            p1 += 2 * dy
            p2 += 2 * dx
            ListOfPoints.append([x1, y1, z1])
    return ListOfPoints

In [3]:
def get_blocks_id_list(loc_dict):
    """
    Given a location dictionary whose values are float coordinates,
    get the blocks ids these coordinates are in.
    :param loc_dict:
    :return:
    """
    loc_list = np.unique(
        [
            (coords - coords % 1).astype(int_dtype) for coords in loc_dict.values()
        ],
        axis=0
    )
    return loc_list

def get_blocks_id_from_centroids(centroids_list):
    """
    Given a list whose values are block centroids,
    get the corresponding block ids these coordinates are in. 
    """
    loc_list = np.unique(
        [
            (coords - coords % 1).astype(int_dtype) for coords in centroids_list
        ], 
        axis=0
    )
    return loc_list 

In [4]:
def spawn_goal(distance, agent_type, drone_centroid=np.array([5.5, 0.5, 5.5])):
    """
    can be replaced with (self).
    only accept 1 goal. distance and agent type can be accessed via self.
    :param distance: a floating number indicating distance between.
    :param agent_type:
    :param drone_centroid:
    :return: a goal dict whose key is goal agent_type id, and a goal coord list.
    """
    # goal would be randomly placed within this sphere around the centroid

    # get the goal agent_ids
    goal_id = [i for i in range(num_agents) if agent_type[i] == 0]
    assert len(goal_id)==1
    goal_dict = {}
    distance = np.array([0, 1, 0]) * distance
    place_radius = 0.4
    displacement = np.random.random(3) * place_radius
    # print(f"displacement: {displacement}")
    goal_dict[goal_id[0]] = drone_centroid + distance + displacement
    return goal_dict

In [5]:
def spawn_drones(num_drones, agent_type, centroid=np.array([5.5, 0.5, 5.5])):
    """
    can be replaced with (self).
    num_drones and agent_type can be accessed via self.
    :return: a drones dict whose keys are drone agent type id, and a drones coord list.
    """
    # spacing parameters, determine the initial distance between drones
    spacing = float_dtype(0.25)
    # actually, the coordinate in centroid is indicating the block number
    # we need to stagger it to get the block centroid coords
    # print(centroid)
    even_rule = np.array([
        [-1, 0, 0], [1, 0, 0],
        [0, 0, -1], [0, 0, 1],
        [-1, 0, 1], [1, 0, -1],
        [1, 0, 1], [-1, 0, -1]
    ], dtype=int_dtype)

    # get the drone id
    drones_id = [i for i in range(num_agents) if agent_type[i] == 1]
    assert len(drones_id)==num_drones

    drones_loc = {}
    if num_drones == 1:
        drones_loc[drones_id[0]] = centroid
    elif num_drones % 2 == 0: # even number of drones
        for _id in range(num_drones):
            # print("step:", even_rule[drone_id % len(even_rule)])
            # print("spacing:", spacing)
            # print("iter:", math.ceil(drone_id / len(even_rule)))
            update = even_rule[_id % len(even_rule)] * spacing * math.ceil((_id + 1) / len(even_rule))
            # print("update: ", update, "type: ", type(update))
            drones_loc[drones_id[_id]] = (
                centroid + update
            )
    else:
        # odd number of drones
        drones_loc[drones_id[0]] = centroid
        for _id in range(num_drones - 1):
            update = even_rule[_id % len(even_rule)] * spacing * math.ceil((_id
              + 1)/ len(even_rule))
            drones_loc[drones_id[_id + 1]] = (
                centroid + update
            )

    return drones_loc

In [6]:
def spawn_obstacles(num_obstacles, agent_type, goal_loc_centroid, drones_loc_centroid=np.array([5.5, 0.5, 5.5])):
    """

    :param num_obstacles:
    :param agent_type:
    :param goal_loc_centroid:
    :param drones_loc_centroid:
    :return: a obstacles dict whose keys are obstacles agent type id, and a obstacles coord list
    """
    # get the occupied blocks of drones and goal
    # 3 by 3 blocks around drones centroid and 3 by 3 blocks around goal centroid are reserved.
    env_size = 10
    env_max_height = 20
    env_min_height = 5 # no spawning of obstacles below this height
  
    stagger = np.array([0.5, 0.5, 0.5], dtype=float_dtype)
    place_radius = 0.4
    displacement = (np.random.rand(num_obstacles, 3) * place_radius).astype(float_dtype)
    # get the obstacles agent ids
    obstacles_id = [i for i in range(num_agents) if agent_type[i] == 0.5]
    assert len(obstacles_id)==num_obstacles

    three_by_three_walk = np.array(
        [
            [0, 0, 0], [1, 0, 0], [-1, 0, 0],
            [0, 0, 1], [1, 0, 1], [-1, 0, 1],
            [0, 0, -1], [1, 0, -1], [-1, 0, -1]
        ]
    ).astype(int_dtype)
    drones_blocks_centroids = [(
            drones_loc_centroid + three_by_three_walk[i])
            for i in range(three_by_three_walk.shape[0]) ]

    goal_blocks_centroids = [(
            goal_loc_centroid + three_by_three_walk[i])
            for i in range(three_by_three_walk.shape[0]) ]
    
    # need to convert these floating points coords into block ids. 
    
    
    occupied_centroids = np.concatenate((
        drones_blocks_centroids, goal_blocks_centroids))
    
    occupied = get_blocks_id_from_centroids(occupied_centroids)

    empty_blocks_id = [
        np.array([x, y, z]).astype(int_dtype)
        for x in range(env_size)
        for y in range(env_size)
        for z in range(env_min_height, env_max_height) if not
        np.any(
            np.all(
                np.array([x, y, z]).astype(int_dtype) == occupied, axis=1
            )
        )
    ]

    empty_indices = np.random.choice(
        len(empty_blocks_id), num_obstacles, replace=False
    )

    obstacles_blocks = [
        empty_blocks_id[empty_indices[i]] for i in range(num_obstacles)
    ]

    obstacles_loc_dict = {
        obstacles_id[_id]: 
        np.array(obstacles_blocks[_id] + stagger + displacement[_id], dtype=float_dtype)
        for _id in range(num_obstacles)
    }
     # obstacles_coord_list = [
     #     obstacles_blocks[i] for i in range(num_obstacles)
     # ]
    return obstacles_loc_dict


In [26]:
def test_generate_city(size, max_height, difficulty_level, sec_num=2):
    """
    return extra stats: (h_mean, h_std,fill_per_sec)
    :param size:
    :param max_height:
    :param difficulty_level:
    :param sec_num:
    :return:
    """
    city_base_map = np.zeros((size, size), dtype=int)

    # the interpolation of mean height and std height
    mean_height_list = np.linspace(0.3, 0.9, 10)
    std_height_list = np.geomspace(2, 0.4, 10)

    # randomly sample pair from above
    idx = random.randint(0, len(mean_height_list)-1)
    mean_height = mean_height_list[idx]
    std_height = std_height_list[idx]

    # based on difficulty level, select the cluster-ness
    fill_per_sec = difficulty_level / 10

    h_mean = int(max_height * mean_height)
    h_std = int(h_mean * std_height)
    cell_size = int(size / sec_num)
    num = int(fill_per_sec * cell_size**2)

    a_min = np.zeros((num), dtype=int)
    a_max = np.full((num), max_height, dtype=int)

    for i in range(sec_num):
        for j in range(sec_num):
            building_height = np.clip(np.random.normal(h_mean, h_std, num), a_min, a_max)
            height_list = np.concatenate((building_height, np.zeros(cell_size*cell_size - num)))
            np.random.shuffle(height_list)
            block_height = np.reshape(height_list, (cell_size, cell_size))
            city_base_map[i*cell_size: (i+1)*cell_size, j*cell_size:(j+1)*cell_size] = block_height

    return city_base_map, (h_mean, h_std,fill_per_sec)

In [8]:
def get_buildings_dict(cbm):
    env_size = 10
    b_dict = {}
    for x in range(env_size):
        for y in range(env_size):
            if cbm[x, y]:
                b_dict[(x, y)] = cbm[x, y]
    return b_dict

### Still WIP...

In [ ]:
def spawn_and_generate_city(
        size,
        max_height,
        difficulty_level,
        is_directly_obs,
        distance,
        agent_type,
        num_drones,
        num_obstacles,
        sec_num=2):
    
    city_base_map = np.zeros((size, size), dtype=int)
    env_size = 10

    # the interpolation of mean height and std height
    mean_height_list = np.linspace(0.3, 0.9, 10)
    std_height_list = np.geomspace(2, 0.4, 10)
    
    # randomly sample pair from above
    idx = random.randint(0, len(mean_height_list)-1)
    mean_height = mean_height_list[idx]
    std_height = std_height_list[idx]
    
    # based on difficulty level, select the cluster-ness
    fill_per_sec = difficulty_level / 10
    
    h_mean = int(max_height * mean_height)
    h_std = int(h_mean * std_height)
    cell_size = int(size / sec_num)
    num = int(fill_per_sec * cell_size**2)

    a_min = np.zeros(num, dtype=int)
    a_max = np.full(num, max_height, dtype=int)

    # spawn the drones formation around the centroid of block (5, 0, 5), using default
    drones_loc_dict = spawn_drones(num_drones, agent_type)

    # spawn the goal location, using default block centroid of (5, 0, 5)
    # the distance here is a floating point value.
    distance = distance * env_size
    goal_loc_dict = spawn_goal(distance, agent_type)
    goal_loc = goal_loc_dict[num_agents - 1]

    obstacles_loc_dict = spawn_obstacles(num_obstacles, agent_type, goal_loc)

    # convert the float coordinates into block ids
    drones_blocks_id = get_blocks_id_list(drones_loc_dict)
    obstacles_blocks_id = get_blocks_id_list(obstacles_loc_dict)
    goal_blocks_id = get_blocks_id_list(goal_loc_dict)

    # get the blocks that in the line of sight between drones and goal.
    # make use of Bresenham3D algorithm, and we only care about the drone centroid.
    # drone centroid should be a class attribute
    drones_centroid = np.array([5, 0, 5]).astype(int_dtype)
    gx, gy, gz = goal_loc[0]
    d_x, d_y, d_z = drones_centroid
    blk_sight_list = Bresenham3D(gx, gy, gz, d_x, d_y, d_z)

    occupied_blocks_list = np.concatenate(
        drones_blocks_id, obstacles_blocks_id, goal_blocks_id
    )

    envelope = np.full((10, 10), 20, dtype=int_dtype) # maximum attainable heights

    # first just generate
    for i in range(sec_num):
        for j in range(sec_num):
            building_height = np.clip(np.random.normal(h_mean, h_std, num), a_min, a_max)
            height_list = np.concatenate((building_height, np.zeros(cell_size*cell_size - num)))
            np.random.shuffle(height_list)
            block_height = np.reshape(height_list, (cell_size, cell_size))
            city_base_map[i*cell_size: (i+1)*cell_size, j*cell_size:(j+1)*cell_size] = block_height

    # TODO: how many blocks must be removed/added at each locs - hard quotas map
    # TODO: just perform the hard update
    # check the occupied location

    # put the occ list inside
    for (x, y, z) in occupied_blocks_list:
        envelope[x, y] = z - 1

    # depending on path_clr:
    # if path should be clear, then add path blocks as well
    # else, put in negative values indicating current difference
    if is_directly_obs:
        for (x, y, z) in blk_sight_list:
            envelope[x, y] = z - 1
    else:
        for (x, y, _) in path:
            city_base_map[x, y] = 5 # just set the height first!

    # perform the hard update
    city_base_map = np.where(city_base_map < envelope, city_base_map, envelope)

    return city_base_map

In [9]:
def mayavi_render_point(p_):
    x_, y_, z_ = np.transpose(p_)
    mayavi.mlab.points3d(
        x_, y_, z_, color=(1, 0, 0), scale_factor=0.05
    )
    mayavi.mlab.show()

def mayavi_render_city(data):
    xx, yy, zz = np.where(data==1)
    mayavi.mlab.points3d(
        xx, yy, zz, mode="cube", color=(0, 1, 0), scale_factor=1
    )
    mayavi.mlab.show()

def mayavi_render_all(data, p_):
    """
    render city, as well as drones, goal, and obstacles with different colors.
    :param data: city bool maps
    :param p_: expected three un-transposed coords lists
    :return: none.
    """
    x, y = np.mgrid[0.5:9.5:10j, 0.5:9.5:10j]
    mlab.barchart(x, y, data, colormap='inferno')
    mlab.vectorbar()

    a_, b_, c_ = p_

    x1, y1, z1 = np.transpose(a_)
    mayavi.mlab.points3d(
        x1, y1, z1, color=(1, 0, 0), scale_factor=0.15
    )
    
#     245, 66, 182
    x2, y2, z2 = np.transpose(b_)
    mayavi.mlab.points3d(
        x2, y2, z2, color=(245/255, 66/255, 182/255), scale_factor=0.15
    )

    x3, y3, z3 = np.transpose(c_)
    mayavi.mlab.points3d(
        x3, y3, z3, color=(1, 1, 0.25), scale_factor=0.15
    )
    mayavi.mlab.show()

## Integrated testing

In [27]:
num_drones = 8
num_goals = 1
num_obstacles = 10
num_agents = 19

drone_set = np.random.choice(
    np.arange(num_agents - 1), num_drones, replace=False
)
agent_type = {}
drones = {}
obstacles = {}
goals = {}
# similarly, only care about the drones and obstacles
for agent_id in range(num_agents - 1):
    if agent_id in set(drone_set):
        agent_type[agent_id] = 1 # drones
        drones[agent_id] = True
    else:
        agent_type[agent_id] = 1 / 2 # obstacles
        obstacles[agent_id] = True

# we append the goal agent id and info at last
agent_type[num_agents-1] = 0 # goal
goals[num_agents] = True

print(f"agent_type: {agent_type}\n")
print(f"drones set: {drone_set}\n")
print(f"drones: {drones}\n")
print(f"obstacles: {obstacles}\n")
print(f"goals:{goals}\n")

agent_type: {0: 0.5, 1: 0.5, 2: 0.5, 3: 0.5, 4: 0.5, 5: 1, 6: 0.5, 7: 1, 8: 1, 9: 0.5, 10: 0.5, 11: 1, 12: 0.5, 13: 0.5, 14: 1, 15: 1, 16: 1, 17: 1, 18: 0}

drones set: [ 7 15  5 14  8 17 11 16]

drones: {5: True, 7: True, 8: True, 11: True, 14: True, 15: True, 16: True, 17: True}

obstacles: {0: True, 1: True, 2: True, 3: True, 4: True, 6: True, 9: True, 10: True, 12: True, 13: True}

goals:{19: True}



In [28]:
o_dict = spawn_obstacles(num_obstacles, agent_type, np.array([5, 3, 5]))
d_dict = spawn_drones(num_drones, agent_type)
d_blk_centroid = np.array([5, 0, 5]).astype(int_dtype)
dis = 2.7
g_dict = spawn_goal(dis, agent_type)

o_list = get_blocks_id_list(o_dict)
d_list = get_blocks_id_list(d_dict)
g_list = get_blocks_id_list(g_dict)

# print(o_list)
# print(len(o_list))
# print(d_list)
# print(len(d_list))
# print(g_list)

occ_list = np.concatenate((o_list, d_list, g_list))
gx, gy, gz = g_list[0]
d_centroid = np.array([5, 0, 5]).astype(int_dtype)
d_x, d_y, d_z = d_centroid
path = Bresenham3D(gx, gy, gz, d_x, d_y, d_z)
# print(d_dict)
# mayavi_render_point(d_list)

In [29]:
print(o_dict)

{0: array([ 7.8181906,  7.5887346, 11.758498 ], dtype=float32), 1: array([ 9.639224,  8.64435 , 18.756952], dtype=float32), 2: array([ 7.7834244,  3.7399988, 15.611481 ], dtype=float32), 3: array([ 4.6871   ,  6.731406 , 11.7770405], dtype=float32), 4: array([ 0.7649697,  4.6146865, 17.71198  ], dtype=float32), 6: array([1.7621695, 3.6898296, 7.536472 ], dtype=float32), 9: array([ 2.6311524,  9.852562 , 19.582708 ], dtype=float32), 10: array([ 8.540976,  4.715695, 10.77076 ], dtype=float32), 12: array([ 7.7133102,  9.588949 , 17.684727 ], dtype=float32), 13: array([ 5.7574534,  6.8934593, 13.763162 ], dtype=float32)}


#### Checking hard quotas and get the update maps

In [30]:
c_b_m, stats= test_generate_city(10, 20, 5)
print(c_b_m)
path_clr = True

h_q_m = np.zeros((10, 10), dtype=int_dtype)
for (x, y, z) in occ_list:
    x_ = x
    y_ = y
    print(f"occ: checking loc:{x, y}, occ: {z}, have: {c_b_m[x_, y_]}")
    h_q_m[x_, y_] = \
        (z - (c_b_m[x_, y_] + 1)) * (c_b_m[x_, y_] + 1 - z > 0)

# path clr for clear line of sight
# first check the removed, denoted as negative entries in hard quota
# building height < occ height
for (x, y, z) in path:
    x_ = x
    y_ = y
    print(f"path: checking loc:{x, y}, have: {c_b_m[x_, y_]}")
    if path_clr:
        h_q_m[x_, y_] = \
            (5 - (c_b_m[x_, y_] + 1)) * (c_b_m[x_, y_] + 1 > 5)
    else:
        h_q_m[x_, y_] = (5 - (c_b_m[x_, y_])) * (c_b_m[x_, y_] < 5)
print(h_q_m)

[[14  0  0 11  1  0  0 14  4  0]
 [11  0 20  0  0  6  0  0  0 20]
 [20  6  0 20  0  0  0  0 12 20]
 [ 0 18  0  0  0  0  0 12 17 20]
 [ 4  1  0 20  0 16 13  0 11  0]
 [16  0 13  0  0  0 20 20  0  4]
 [ 0 20 18  0  5  0 10 18 19  0]
 [ 3  0  0  5  0 20 19  0 14 10]
 [ 4  0 13 11  3  0  0  0  0  0]
 [ 0  0  0  0  8  0  0 20  0  0]]
occ: checking loc:(0, 4), occ: 17, have: 1
occ: checking loc:(1, 3), occ: 7, have: 0
occ: checking loc:(2, 9), occ: 19, have: 20
occ: checking loc:(4, 6), occ: 11, have: 13
occ: checking loc:(5, 6), occ: 13, have: 20
occ: checking loc:(7, 3), occ: 15, have: 5
occ: checking loc:(7, 7), occ: 11, have: 0
occ: checking loc:(7, 9), occ: 17, have: 10
occ: checking loc:(8, 4), occ: 10, have: 3
occ: checking loc:(9, 8), occ: 18, have: 0
occ: checking loc:(5, 0), occ: 5, have: 16
occ: checking loc:(5, 3), occ: 5, have: 0
path: checking loc:(5, 3), have: 0
path: checking loc:(5, 2), have: 13
path: checking loc:(5, 1), have: 0
path: checking loc:(5, 0), have: 16
[[  0   0

In [31]:
print(stats) # h_mean, h_std, fill_per_sec

(15, 8, 0.5)


#### Perform a hard update and get the new map

In [32]:
c_b_m_hard = c_b_m + h_q_m
print(c_b_m_hard)
# mlab.barchart(c_b_m_hard, colormap='BuPu')
# mlab.vectorbar()
# mlab.show()

[[14  0  0 11  1  0  0 14  4  0]
 [11  0 20  0  0  6  0  0  0 20]
 [20  6  0 20  0  0  0  0 12 18]
 [ 0 18  0  0  0  0  0 12 17 20]
 [ 4  1  0 20  0 16 10  0 11  0]
 [ 4  0  4  0  0  0 12 20  0  4]
 [ 0 20 18  0  5  0 10 18 19  0]
 [ 3  0  0  5  0 20 19  0 14 10]
 [ 4  0 13 11  3  0  0  0  0  0]
 [ 0  0  0  0  8  0  0 20  0  0]]


#### Collect coordinates from location dictionary

In [ ]:
# test and visualize
d_coords = np.array([coord for coord in d_dict.values()])
o_coords = np.array([coord for coord in o_dict.values()])
g_coords = np.array([coord for coord in g_dict.values()])

# c_bool_m_hard = base_map2bool_maps(c_b_m_hard, 10, 20)

# mayavi_render_all(c_b_m_hard, (d_coords,o_coords,g_coords))

#### Check for soft quotas

In [45]:
# check the soft quotas
# 10 should be replaced with env_size
s_add_map = np.zeros((10, 10),dtype=int_dtype)
s_rem_map = np.zeros((10, 10),dtype=int_dtype)

# calculate statistics
fps_now = np.count_nonzero(c_b_m_hard) / (10 * 10)
h_m_now = c_b_m_hard.mean()
h_s_now = c_b_m_hard.std()

# get the expected statistics
h_m_exp, h_s_exp, fps_exp = stats

# calculate the difference percentage
fps_diff = (fps_now - fps_exp) / fps_now
h_m_diff = (h_m_now - h_m_exp) / h_m_now
h_s_diff = (h_s_now - h_s_exp) / h_s_now
print("fps diff: {:.3f}, h_m diff: {:.3f}, h_s diff: {:.3f}".format(fps_diff, h_m_diff, h_s_diff))

fps diff: -0.064, h_m diff: -1.632, h_s diff: -0.070


In [48]:
# calculate the soft quotas
# check the occ
for (x, y, z) in occ_list:
    s_rem_map[x, y] = c_b_m_hard[x, y]
    s_add_map[x, y] = z - c_b_m_hard[x, y] - 1

# check the path
for (x, y, z) in path:
    if path_clr: # buildings limited to below height of 5
        s_rem_map[x, y] = c_b_m_hard[x, y]
        s_add_map[x, y] = z - c_b_m_hard[x, y] - 1
    else: # buildings should be taller than height of 5, which is already guaranteed. only below height limits.
        s_add_map[x, y] = 18 - c_b_m_hard[x, y]

# check the other empty space, Nope!!!!
print("soft add map: \n", s_add_map)
print("soft rem map: \n", s_rem_map)

soft add map: 
 [[ 0  0  0  0 15  0  0  0  0  0]
 [ 0  0  0  6  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  4  0  4  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  9  0  0  0 10  0  6]
 [ 0  0  0  0  6  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 17  0]]
soft rem map: 
 [[ 0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 18]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 10  0  0  0]
 [ 4  0  4  0  0  0 12  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  5  0  0  0  0  0 10]
 [ 0  0  0  0  3  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]]


In [49]:
print(c_b_m_hard)

[[14  0  0 11  1  0  0 14  4  0]
 [11  0 20  0  0  6  0  0  0 20]
 [20  6  0 20  0  0  0  0 12 18]
 [ 0 18  0  0  0  0  0 12 17 20]
 [ 4  1  0 20  0 16 10  0 11  0]
 [ 4  0  4  0  0  0 12 20  0  4]
 [ 0 20 18  0  5  0 10 18 19  0]
 [ 3  0  0  5  0 20 19  0 14 10]
 [ 4  0 13 11  3  0  0  0  0  0]
 [ 0  0  0  0  8  0  0 20  0  0]]


In [50]:
# thinking in height limit ways
# checking and updating can be done without laboriously accessing arrays entry by entry
new_ceilings = np.full((10, 10), 20, dtype=int_dtype) # maximum attainable heights

# update them!
# put the occ list inside
for (x, y, z) in occ_list:
    new_ceilings[x, y] = z - 1

# depending on path_clr:
# if path should be clear, then add path blocks as well
# else, put in negative values indicating current difference
if path_clr:
    for (x, y, z) in path:
        new_ceilings[x, y] = z - 1
else:
    for (x, y, _) in path:
        c_b_m_hard[x, y] = 5 # just set the height first!

print(new_ceilings)


[[20 20 20 20 16 20 20 20 20 20]
 [20 20 20  6 20 20 20 20 20 20]
 [20 20 20 20 20 20 20 20 20 18]
 [20 20 20 20 20 20 20 20 20 20]
 [20 20 20 20 20 20 10 20 20 20]
 [ 4  4  4  4 20 20 12 20 20 20]
 [20 20 20 20 20 20 20 20 20 20]
 [20 20 20 14 20 20 20 10 20 16]
 [20 20 20 20  9 20 20 20 20 20]
 [20 20 20 20 20 20 20 20 17 20]]


In [52]:
print(c_b_m)

[[14  0  0 11  1  0  0 14  4  0]
 [11  0 20  0  0  6  0  0  0 20]
 [20  6  0 20  0  0  0  0 12 20]
 [ 0 18  0  0  0  0  0 12 17 20]
 [ 4  1  0 20  0 16 13  0 11  0]
 [16  0 13  0  0  0 20 20  0  4]
 [ 0 20 18  0  5  0 10 18 19  0]
 [ 3  0  0  5  0 20 19  0 14 10]
 [ 4  0 13 11  3  0  0  0  0  0]
 [ 0  0  0  0  8  0  0 20  0  0]]


In [57]:
# perform the hard update
c_b_m_1 = np.where(c_b_m < new_ceilings, c_b_m, new_ceilings)

In [58]:
new_ceilings - c_b_m_1 # possible room for growth, i.e. the soft add maps

array([[ 6, 20, 20,  9, 15, 20, 20,  6, 16, 20],
       [ 9, 20,  0,  6, 20, 14, 20, 20, 20,  0],
       [ 0, 14, 20,  0, 20, 20, 20, 20,  8,  0],
       [20,  2, 20, 20, 20, 20, 20,  8,  3,  0],
       [16, 19, 20,  0, 20,  4,  0, 20,  9, 20],
       [ 0,  4,  0,  4, 20, 20,  0,  0, 20, 16],
       [20,  0,  2, 20, 15, 20, 10,  2,  1, 20],
       [17, 20, 20,  9, 20,  0,  1, 10,  6,  6],
       [16, 20,  7,  9,  6, 20, 20, 20, 20, 20],
       [20, 20, 20, 20, 12, 20, 20,  0, 17, 20]])